In [1]:
# Fake News Classifier Using LSTM

In [2]:
import pandas as pd

In [4]:
df = pd.read_csv('train.csv')

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# Shape of the dataset
df.shape

(20800, 5)

In [11]:
# Drop this null values
df = df.dropna()

In [12]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
# Get the independent features
X = df.drop('label',axis=1)

In [16]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [17]:
# Get the dependent feature
Y = df['label']

In [18]:
Y.head()

0    1
1    0
2    1
3    1
4    1
Name: label, dtype: int64

In [19]:
X.shape

(18285, 4)

In [20]:
Y.shape

(18285,)

In [23]:
# Importing Tensorflow
import tensorflow as tf
print(tf.__version__)

2.15.0


In [68]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [69]:
# Vocabulary size
voc_size = 5000

In [70]:
# One-Hot Representation
messages = X.copy()

In [71]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [31]:
# Displaying all the messages
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [72]:
# Reset the index
messages.reset_index(inplace = True)

In [73]:
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [74]:
# Applying NLTK to remove stopwords.
import nltk
import re
from nltk.corpus import stopwords

In [75]:
# Download stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kaushik\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [76]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer # Uses for stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [77]:
# Seeing the corpus
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [78]:
# Applying One-Hot Representation
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr

[[4142, 4115, 1780, 2801, 17, 2619, 1053, 35, 4065, 2791],
 [1924, 391, 3605, 2104, 3891, 2074, 4352],
 [2224, 583, 30, 1193],
 [2382, 1161, 1518, 3414, 3101, 4326],
 [894, 3891, 479, 1199, 4258, 4452, 3891, 773, 4463, 3255],
 [4949,
  3131,
  54,
  4184,
  556,
  3647,
  2101,
  4446,
  1188,
  2617,
  1932,
  4719,
  2658,
  350,
  4352],
 [1332, 1129, 4776, 2798, 537, 425, 537, 4795, 1475, 1881, 2967],
 [326, 1307, 99, 3300, 2007, 2037, 3647, 29, 1475, 1881, 2967],
 [150, 2439, 4254, 3613, 4028, 619, 1877, 1499, 3647, 3274],
 [2728, 2141, 747, 4427, 4812, 162, 323, 2312],
 [3662, 4538, 821, 4973, 2725, 4130, 4275, 4881, 4815, 4819, 3699],
 [3414, 356, 17, 619, 3647, 2007],
 [529, 3855, 145, 1528, 4586, 1953, 512, 4160, 1938],
 [3077, 1291, 4191, 3733, 3377, 4775, 1861, 1475, 1881, 2967],
 [1701, 1260, 1720, 2892, 738, 1475, 1881, 2967],
 [996, 594, 3044, 2508, 983, 174, 2469, 4995, 2802, 248],
 [2494, 3150, 391],
 [640, 1668, 4079, 1051, 3647, 4433, 3609, 4352],
 [2461, 3792, 3605, 

In [79]:
# Embedding representation
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ...,   35, 4065, 2791],
       [   0,    0,    0, ..., 3891, 2074, 4352],
       [   0,    0,    0, ...,  583,   30, 1193],
       ...,
       [   0,    0,    0, ..., 1475, 1881, 2967],
       [   0,    0,    0, ...,  247, 2979, 3583],
       [   0,    0,    0, ...,  289, 3489, 2450]])

In [80]:
# Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

In [81]:
# Compiling the Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_2 (LSTM)               (None, 200)               192800    
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 393001 (1.50 MB)
Trainable params: 393001 (1.50 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [83]:
len(embedded_docs), Y.shape

(18285, (18285,))

In [84]:
# Converting to array
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [85]:
# Shape of final array
X_final.shape, Y_final.shape

((18285, 20), (18285,))

In [86]:
# Applying the train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

In [87]:
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 24ms/step - loss: 0.3255 - accuracy: 0.8433 - val_loss: 0.1942 - val_accuracy: 0.9210
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1310 - accuracy: 0.9485 - val_loss: 0.2047 - val_accuracy: 0.9218
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0921 - accuracy: 0.9659 - val_loss: 0.2232 - val_accuracy: 0.9168
Epoch 4/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0611 - accuracy: 0.9794 - val_loss: 0.2762 - val_accuracy: 0.9168
Epoch 5/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0361 - accuracy: 0.9878 - val_loss: 0.3315 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0204 - accuracy: 0.9936 - val_loss: 0.3642 - val_accuracy: 0.9130
Epoch 7/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0130 - accuracy: 0.9960 - val_loss: 0.5743 - val_accuracy: 0.9127

In [88]:
# Performance metrics and accuracy
y_pred = model.predict(X_test)
y_pred

189/189 [==============================] - 1s 5ms/step


array([[9.9999744e-01],
       [2.0125976e-02],
       [4.2845382e-05],
       ...,
       [9.1938819e-05],
       [9.9759722e-01],
       [9.9999440e-01]], dtype=float32)

In [89]:
y_pred = np.where(y_pred > 0.5, 1, 0)
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [90]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, y_pred)

array([[3132,  287],
       [ 285, 2331]], dtype=int64)

In [91]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, y_pred)

0.9052195526097763

In [104]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.91      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.91      0.91      0.91      6035
